<a href="https://colab.research.google.com/github/asmajabr/FraudDetectionML/blob/master/FraudDetectionModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import os
cwd = os.getcwd()
print (cwd)
!ls -lah

/content
total 20K
drwxr-xr-x 1 root root 4.0K Jul  1 05:21 .
drwxr-xr-x 1 root root 4.0K Jul  1 05:16 ..
drwxr-xr-x 1 root root 4.0K Jun 26 16:26 .config
drwx------ 4 root root 4.0K Jul  1 05:21 gdrive
drwxr-xr-x 1 root root 4.0K Jun 26 16:26 sample_data


In [4]:
cwd = os.getcwd()
print (cwd)
!ls -lah

/content
total 20K
drwxr-xr-x 1 root root 4.0K Jun 30 09:02 .
drwxr-xr-x 1 root root 4.0K Jun 30 08:58 ..
drwxr-xr-x 1 root root 4.0K Jun 26 16:26 .config
drwx------ 4 root root 4.0K Jun 30 09:02 gdrive
drwxr-xr-x 1 root root 4.0K Jun 26 16:26 sample_data


In [5]:
cwd = os.getcwd()
print (cwd)
!ls -lah

/content
total 20K
drwxr-xr-x 1 root root 4.0K Jun 30 09:02 .
drwxr-xr-x 1 root root 4.0K Jun 30 08:58 ..
drwxr-xr-x 1 root root 4.0K Jun 26 16:26 .config
drwx------ 4 root root 4.0K Jun 30 09:02 gdrive
drwxr-xr-x 1 root root 4.0K Jun 26 16:26 sample_data


In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')

url = "/content/gdrive/My Drive/creditcard.csv"

creditcard = pd.read_csv(url)

creditcard.head()


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Modeling Part 2: RandomForestClassifier
1. Use the Imbalanced Data Directly in RandomForestClassifier
2. Create Over-sampling data and Fit the model
3. RandomForestClassifier with class_weight
4. Self-defined Score and GridSearchCV of hyperparameter




Conclusion:
As expected, use the imbalanced data is not a good way. The performance is the worst compared to using over-sampling or class weights
Use imbalanced data, RandomForestClassifier result is better than LogisticRegression.
If we can custom a good loss function, the model performance will be better: here the customed loss function performance is better than roc_auc scoring function.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, Normalizer, scale
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix, log_loss, auc, roc_curve, roc_auc_score, recall_score, precision_recall_curve
from sklearn.metrics import make_scorer, precision_score, fbeta_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedShuffleSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)

seed = 999


creditcard.columns = [x.lower() for x in creditcard.columns]
creditcard.rename(columns = {'class': 'fraud'}, inplace = True)

In [6]:
creditcard.drop(columns = 'time', inplace = True)

# Normalize the 'amount' column
scaler = StandardScaler()
creditcard['amount'] = scaler.fit_transform(creditcard['amount'].values.reshape(-1, 1))
# creditcard.drop(columns = 'amount', inplace = True)
creditcard['amount']
X = creditcard.iloc[:, :-1]
y = creditcard.iloc[:, -1]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = .33, stratify = y, random_state = seed)

For non-fraud transactions, the average amount is 88. For fraud transactions, the average amount is 122. So, in average there will be 122 loss for a fraud. Suppose for each transaction, the company can get 2% transaction fee. That is, the average is 88 * 2% = 1.76.

That means: if we predict a non-fraud as fraud, we might loss 1.76. However, if we miss to detect a fraud transaction, we will loss about 122.

Usually for imbalanced data, we can try:

1. Collect more data (which not work here since the data is given)
2. Down-Sampling or Over-Sampling to get balanced samples
3. Change the Thresholds to adjust the prediction
4. Assign class weights for the low rate class

Here we will try 4 different ways and compare their results:

2.1. Do nothing, use original data to model
2.2. Do Over-Sampling, use the over-sampled data to model
2.3. Assigning sample weights in RandomForestClassifier
2.4. Use customed loss function

1. Use the Imbalanced Data Directly in RandomForestClassifier

In [ ]:
X = creditcard.iloc[:, :-1]
y = creditcard.iloc[:, -1]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = .33, stratify = y, random_state = seed)

estimator = RandomForestClassifier(random_state=0, warm_start = True, n_jobs = 70)

rf_tuned_parameters = {"max_depth": [10, 20, 50, 100], 'n_estimators': [50, 100, 200, 500], 'min_samples_leaf': [10, 20, 50]}

cv_grid = GridSearchCV(estimator, param_grid = rf_tuned_parameters, scoring = 'roc_auc', verbose = 5) # 'recall', my_score
cv_grid.fit(Xtrain, ytrain)

# print cv_grid.cv_results_

best_parameters = cv_grid.best_estimator_.get_params()

# for param_name in sorted(rf_tuned_parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))

pred_test = cv_grid.predict(Xtest)
print(recall_score(ytest, pred_test))     # 0.65
print(precision_score(ytest, pred_test))  # 0.85
print(roc_auc_score(ytest, pred_test))    # 0.83
print("confustion matrix on validation data: \n" + str(confusion_matrix(ytest, pred_test)))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] max_depth=10, min_samples_leaf=10, n_estimators=50 ..............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


2. Create Over-sampling data and Fit the model

In [8]:
oversample_ratio = sum(ytrain == 0) / sum(ytrain == 1)  # size to repeat y == 1
# repeat the positive data for X and y
ytrain_pos_oversample = pd.concat([ytrain[ytrain==1]] * oversample_ratio, axis = 0)
Xtrain_pos_oversample = pd.concat([Xtrain.loc[ytrain==1, :]] * oversample_ratio, axis = 0)
# concat the repeated data with the original data together
ytrain_oversample = pd.concat([ytrain, ytrain_pos_oversample], axis = 0).reset_index(drop = True)
Xtrain_oversample = pd.concat([Xtrain, Xtrain_pos_oversample], axis = 0).reset_index(drop = True)

ytrain_oversample.value_counts(dropna = False, normalize = True)   # 50:50

estimator = RandomForestClassifier(random_state=0, warm_start = True, n_jobs = 70)

rf_tuned_parameters = {"max_depth": [10, 20, 50, 100], 'n_estimators': [50, 100, 200, 500], 'min_samples_leaf': [10, 20, 50]}

cv_grid = GridSearchCV(estimator, param_grid = rf_tuned_parameters, scoring = 'roc_auc', verbose = 5) # 'recall', my_score
cv_grid.fit(Xtrain_oversample, ytrain_oversample)

# print cv_grid.best_params_
# print cv_grid.cv_results_

best_parameters = cv_grid.best_estimator_.get_params()

# for param_name in sorted(rf_tuned_parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))

pred_test = cv_grid.predict(Xtest)
print(recall_score(ytest, pred_test))     # 0.83
print(precision_score(ytest, pred_test))  # 0.83
print(roc_auc_score(ytest, pred_test))    # 0.92
print("\n confustion matrix on validation data: \n" + str(confusion_matrix(ytest, pred_test)))

TypeError: ignored

3. RandomForestClassifier with class_weight

---
Rather than over-sampling, we can assign more weights to the lower rate class.


In [1]:
X = creditcard.iloc[:, :-1]
y = creditcard.iloc[:, -1]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = .33, stratify = y, random_state = seed)

positive_weight = sum(ytrain == 0) / sum(ytrain == 1)  # size to repeat y == 1

estimator = RandomForestClassifier(random_state=0, class_weight = {0 : 1, 1 : positive_weight}, warm_start = True, n_jobs = 70)

rf_tuned_parameters = {"max_depth": [10, 20, 50, 100], 'n_estimators': [50, 100, 200, 500], 'min_samples_leaf': [10, 20, 50]}

cv_grid = GridSearchCV(estimator, param_grid = rf_tuned_parameters, scoring = 'roc_auc', verbose = 5) # 'recall', my_score
cv_grid.fit(Xtrain, ytrain)

# print cv_grid.cv_results_

best_parameters = cv_grid.best_estimator_.get_params()

# for param_name in sorted(rf_tuned_parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))

pred_test = cv_grid.predict(Xtest)
print(recall_score(ytest, pred_test))     #  0.85
print(precision_score(ytest, pred_test))  #  0.81
print(roc_auc_score(ytest, pred_test))    #  0.92
print("\n confustion matrix on validation data: \n" + str(confusion_matrix(ytest, pred_test)))

NameError: ignored

4. Self-defined Score and GridSearchCV of hyperparameter:
Since the loss from frauds and false predicted frauds are different for us. We will define a function to re-weight the effects by average loss from missing predicted frauds and falsely predicted frauds.

In [ ]:
def scoring(ground_truth, predictions):
    '''
    based on results above about the average loss from false positive and false negative predictions.
    '''
    cmatrix = confusion_matrix(ground_truth, predictions)
    fp = cmatrix[0, 1]
    fn = cmatrix[1, 0]
    return  fn * 122 + fp * 1.76

wt_loss_score = make_scorer(scoring, greater_is_better = False)

oversample_ratio = sum(ytrain == 0) / sum(ytrain == 1)  # size to repeat y == 1
# repeat the positive data for X and y
ytrain_pos_oversample = pd.concat([ytrain[ytrain==1]] * oversample_ratio, axis = 0)
Xtrain_pos_oversample = pd.concat([Xtrain.loc[ytrain==1, :]] * oversample_ratio, axis = 0)
# concat the repeated data with the original data together
ytrain_oversample = pd.concat([ytrain, ytrain_pos_oversample], axis = 0).reset_index(drop = True)
Xtrain_oversample = pd.concat([Xtrain, Xtrain_pos_oversample], axis = 0).reset_index(drop = True)

ytrain_oversample.value_counts(dropna = False, normalize = True)   # 50:50

estimator = RandomForestClassifier(random_state=0, warm_start = True)

rf_tuned_parameters = {"max_depth": [10, 20, 50, 100], 'n_estimators': [50, 100, 200, 500], 
                       'min_samples_leaf': [10, 20, 50]}

cv_grid = GridSearchCV(estimator, param_grid = rf_tuned_parameters, scoring = wt_loss_score, verbose = 5, n_jobs = 70)
cv_grid.fit(Xtrain_oversample, ytrain_oversample)

# print cv_grid.best_params_

pred_test = cv_grid.predict(Xtest)
print(recall_score(ytest, pred_test))     # 0.84
print(precision_score(ytest, pred_test))  # 0.84
print(roc_auc_score(ytest, pred_test))    # 0.92
print("\n confustion matrix on validation data: \n" + str(confusion_matrix(ytest, pred_test)))